In [1]:
%matplotlib inline

import os, re, glob
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join as opj
from tqdm import tqdm_notebook
from ast import literal_eval as make_tuple

sns.set_context('paper')

In [2]:
gsearch = lambda *args: glob.glob(opj(*args))
str_extract = lambda pattern, s: re.search(pattern, s).group(0)

In [5]:
data_dir = '../data/publicLifeLogData'
all_files = gsearch(data_dir, 'lifeLog_bigserver2*', '2*y.txt')
all_files.sort()
print(*all_files[:10], sep='\n')

../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_01January_29_Tuesday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_01January_30_Wednesday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_01January_31_Thursday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_01_Friday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_02_Saturday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_03_Sunday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_04_Monday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_05_Tuesday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_06_Wednesday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_02February_07_Thursday.txt


In [6]:
def date_extract(s):
    
    date_regex = '([0-9]{4})_([0-9]{2})[A-Za-z]+_([0-9]{2})'
    date_search = re.findall(date_regex, s)
    date_str = ''.join(date_search[0])
    date_int = int(date_str)
    
    return date_int

In [7]:
def find_version(file):
    file_date = date_extract(file)

    tmp_ver = ver.copy()
    tmp_ver['lag'] = file_date - tmp_ver['release_date']
    tmp_ver = tmp_ver[tmp_ver['lag'] >= 0]

    file_ver = tmp_ver.loc[tmp_ver['lag'].idxmin()].version
    
    return file_ver

In [8]:
ver_file = '../1_download/outputs/version_history_full.tsv'

# Load file
ver = pd.read_csv(ver_file, sep='\t')

# Clean up release date
ver.release_date = [s.replace('-', '') for s in ver.release_date]
ver.release_date = pd.to_numeric(ver.release_date)

In [9]:
all_dates = [date_extract(f) for f in all_files]
all_dates

[20190129,
 20190130,
 20190131,
 20190201,
 20190202,
 20190203,
 20190204,
 20190205,
 20190206,
 20190207,
 20190208,
 20190209,
 20190210,
 20190211,
 20190212,
 20190213,
 20190214,
 20190215,
 20190216,
 20190217,
 20190218,
 20190219,
 20190220,
 20190221,
 20190222,
 20190223,
 20190224,
 20190225,
 20190226,
 20190227,
 20190228,
 20190301,
 20190302,
 20190303,
 20190304,
 20190305,
 20190306,
 20190307,
 20190308,
 20190309,
 20190310,
 20190311,
 20190312,
 20190313,
 20190314,
 20190315,
 20190316,
 20190317,
 20190318,
 20190319,
 20190320,
 20190321,
 20190322,
 20190323,
 20190324,
 20190325,
 20190326,
 20190327,
 20190328,
 20190329,
 20190330,
 20190331,
 20190401,
 20190402,
 20190403,
 20190404,
 20190405,
 20190406,
 20190407,
 20190408,
 20190409,
 20190410,
 20190411,
 20190412,
 20190413,
 20190414,
 20190415,
 20190416,
 20190417,
 20190418,
 20190419,
 20190420,
 20190421,
 20190422,
 20190423,
 20190424,
 20190425,
 20190426,
 20190427,
 20190428,
 20190429,

In [10]:
start = 20190726

# Check if date is within range
between_dates = lambda d: (d >= start)

file_df = pd.DataFrame({'file': all_files,
                        'date': all_dates})
file_df['include'] = file_df.date.apply(between_dates)
file_df = file_df[file_df.include == True]

file_df.head()

,file,date,include
178,../data/publicLifeLogData/lifeLog_bigserver2.o...,20190726,True
179,../data/publicLifeLogData/lifeLog_bigserver2.o...,20190727,True
180,../data/publicLifeLogData/lifeLog_bigserver2.o...,20190728,True
181,../data/publicLifeLogData/lifeLog_bigserver2.o...,20190729,True
182,../data/publicLifeLogData/lifeLog_bigserver2.o...,20190730,True


In [11]:
from tqdm import tqdm
data_files = file_df['file'].values
data_files

array(['../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_07July_26_Friday.txt',
       '../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_07July_27_Saturday.txt',
       '../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_07July_28_Sunday.txt',
       '../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_07July_29_Monday.txt',
       '../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_07July_30_Tuesday.txt',
       '../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_07July_31_Wednesday.txt',
       '../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_08August_01_Thursday.txt',
       '../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_08August_02_Friday.txt',
       '../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_08August_03_Saturday.txt',
       '../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_08Augus

In [17]:
data_list = []
empty_files = []
for i in tqdm(range(len(data_files))):
    f = data_files[i]
    print(f)
# for f in data_files:
    try:
        tmp_server = str_extract('(?<=lifeLog_)[a-zA-Z0-9]+', f)
        tmp_ver = find_version(f)
        
        tmp_d = pd.read_csv(f, sep =' ', header=None)
        tmp_d.insert(0, 'server', tmp_server)
        tmp_d.insert(0, 'release', tmp_ver)
        data_list.append(tmp_d)

    except:
        empty_files.append(f)

  0%|          | 0/327 [00:00<?, ?it/s]

../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_07July_26_Friday.txt


  1%|          | 3/327 [00:00<02:23,  2.27it/s]

../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_07July_27_Saturday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_07July_28_Sunday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_07July_29_Monday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2019_07July_30_Tuesday.txt


  2%|▏         | 7/327 [00:00<01:18,  4.10it/s]

...

 97%|█████████▋| 318/327 [00:11<00:00, 40.76it/s]

../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_02_Tuesday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_03_Wednesday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_04_Thursday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_05_Friday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_06_Saturday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_07_Sunday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_08_Monday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_09_Tuesday.txt


 99%|█████████▉| 323/327 [00:11<00:00, 37.16it/s]

../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_10_Wednesday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_11_Thursday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_12_Friday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_13_Saturday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_14_Sunday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_15_Monday.txt
../data/publicLifeLogData/lifeLog_bigserver2.onehouronelife.com/2020_06June_16_Tuesday.txt


100%|██████████| 327/327 [00:11<00:00, 27.48it/s]


In [18]:
raw_data = pd.concat(data_list)
raw_data.columns = ['release', 'server', 'event', 'timestamp', 'playerID', 'uniqueID', 'param1', 'param2',
                 'param3', 'param4', 'param5']
raw_data.head()

,release,server,event,timestamp,playerID,uniqueID,param1,param2,param3,param4,param5
0,251,bigserver2,D,1564099205,1806575,91f6003fa6a03933f72b1f95a7002f1f07928deb,age=33.40,M,"(277,-2434)",hunger,pop=67
1,251,bigserver2,D,1564099220,1806488,1a7f24be09f45519e96d4f017b36ad38eb5e329d,age=60.00,M,"(-2516,-1279)",oldAge,pop=66
2,251,bigserver2,D,1564099234,1806629,3dc579b91219e519dfee2f9d7a24aec8e9a83b03,age=19.04,F,"(-1919,-2078)",hunger,pop=65
3,251,bigserver2,D,1564099254,1806530,780359de9275c29b15b65d3cdba5b22bf90169c5,age=47.75,F,"(-2525,-1276)",hunger,pop=64
4,251,bigserver2,D,1564099256,1806662,bcc9ac263a4036f0141ffb04e9224e00443bb791,age=10.92,F,"(277,-2433)",hunger,pop=64


In [26]:
birth_data = raw_data[raw_data['event'] == 'B'].copy()
birth_data = birth_data[['release','playerID', 'timestamp', 'uniqueID', 'param2','param5']]
birth_data = birth_data.rename({'param2': 'birth_location','param5': 'chain'}, axis='columns')
birth_data.head()

,release,playerID,timestamp,uniqueID,birth_location,chain
6,251,1806690,1564099277,610232431bd719fc6ae03c734f0f2569a659d59b,"(-1933,-2118)",chain=45
9,251,1806691,1564099329,14a34de5ef24a092c09f2f726904939a8b7fbd1f,"(-1802,-719)",chain=5
10,251,1806692,1564099361,207e9318d08219318f00fd32e483251f81247dc8,"(-2432,-4839)",chain=10
11,251,1806693,1564099385,52c80bc1fa937e1eceb41ba28610da4c0da196f2,"(-1930,-2023)",chain=44
12,251,1806694,1564099409,265223909fa4f36f07a340f020cafdb74d91615e,"(-2519,-1282)",chain=21


In [32]:
death_data = raw_data[raw_data['event'] == 'D'].copy()
death_data = death_data[['release','playerID', 'timestamp', 'uniqueID', 'param3']]
death_data = death_data.rename({'param3': 'death_location'}, axis='columns')
death_data.head()


,release,playerID,timestamp,uniqueID,death_location
0,251,1806575,1564099205,91f6003fa6a03933f72b1f95a7002f1f07928deb,"(277,-2434)"
1,251,1806488,1564099220,1a7f24be09f45519e96d4f017b36ad38eb5e329d,"(-2516,-1279)"
2,251,1806629,1564099234,3dc579b91219e519dfee2f9d7a24aec8e9a83b03,"(-1919,-2078)"
3,251,1806530,1564099254,780359de9275c29b15b65d3cdba5b22bf90169c5,"(-2525,-1276)"
4,251,1806662,1564099256,bcc9ac263a4036f0141ffb04e9224e00443bb791,"(277,-2433)"


In [34]:
loc_data = birth_data.merge(death_data, on = ['release','playerID','uniqueID'])
loc_data['birth'] = loc_data['birth_location'].apply(make_tuple).apply(np.array)
loc_data['death'] = loc_data['death_location'].apply(make_tuple).apply(np.array)

# Split coordinates (for easier plotting)
loc_data[['birthX', 'birthY']] = pd.DataFrame(loc_data['birth'].tolist(),
                                              index=loc_data.index)   
loc_data[['deathX', 'deathY']] = pd.DataFrame(loc_data['death'].tolist(),
                                              index=loc_data.index)

lifeLog = loc_data[['release','playerID','timestamp_x','timestamp_y','uniqueID','birthX','birthY','deathX','deathY','chain']]


lifeLog.head()

,release,playerID,timestamp_x,timestamp_y,uniqueID,birthX,birthY,deathX,deathY,chain
0,251,1806690,1564099277,1564101819,610232431bd719fc6ae03c734f0f2569a659d59b,-1933,-2118,-1950,-2089,chain=45
1,251,1806691,1564099329,1564100950,14a34de5ef24a092c09f2f726904939a8b7fbd1f,-1802,-719,-1825,-549,chain=5
2,251,1806692,1564099361,1564102961,207e9318d08219318f00fd32e483251f81247dc8,-2432,-4839,-2426,-4832,chain=10
3,251,1806693,1564099385,1564102985,52c80bc1fa937e1eceb41ba28610da4c0da196f2,-1930,-2023,-1946,-2557,chain=44
4,251,1806694,1564099409,1564100461,265223909fa4f36f07a340f020cafdb74d91615e,-2519,-1282,-2518,-1348,chain=21


In [36]:
# turn str under chain into num
digit = lambda x: int(x.split("=")[1])
lifeLog['num_chain'] = lifeLog['chain'].apply(digit)
lifeLogNew = lifeLog.drop(['chain'], axis=1)
lifeLogNew.head()

/Users/dengwenning/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,release,playerID,timestamp_x,timestamp_y,uniqueID,birthX,birthY,deathX,deathY,num_chain
0,251,1806690,1564099277,1564101819,610232431bd719fc6ae03c734f0f2569a659d59b,-1933,-2118,-1950,-2089,45
1,251,1806691,1564099329,1564100950,14a34de5ef24a092c09f2f726904939a8b7fbd1f,-1802,-719,-1825,-549,5
2,251,1806692,1564099361,1564102961,207e9318d08219318f00fd32e483251f81247dc8,-2432,-4839,-2426,-4832,10
3,251,1806693,1564099385,1564102985,52c80bc1fa937e1eceb41ba28610da4c0da196f2,-1930,-2023,-1946,-2557,44
4,251,1806694,1564099409,1564100461,265223909fa4f36f07a340f020cafdb74d91615e,-2519,-1282,-2518,-1348,21


In [ ]:
##plot migration across versions
fig = plt.figure(figsize=(30,30))
version = 301
sns.scatterplot(data = lifeLog[lifeLog['release'] == version], x = 'birthX', y = 'birthY', hue ='timestamp_x',
               palette="Blues",alpha =0.8)

sns.scatterplot(data = lifeLog[lifeLog['release'] == (version+1)], x = 'birthX', y = 'birthY', hue ='timestamp_x',
               palette="Reds",alpha =0.8)
plt.title('version' + str(version) + '-' + str(version+1))

# plt.savefig('plots/update2.png', dpi=100)

In [123]:
##this function plots the birth and death location of each player
def plot_individual_immigration(sub_version):
    loc_subset = lifeLog[lifeLog['release'] == sub_version] ##plot migration for each version
    
    n_data = len(loc_subset)
    
    if n_data > 1: ##seems like it's missing data from some of the versions
        if n_data > 5000:
            a = 0.25
        else:
            a = 1

        fig = plt.figure(figsize=(30,30))
        for i, row in loc_subset.iterrows():
            plt.arrow(row['birthX'], row['birthY'],
                      row['deathX']-row['birthX'],
                      row['deathY']-row['birthY'],
                      alpha=0.2)
        sns.scatterplot(data = loc_subset, x = 'birthX', y = 'birthY', hue ='num_chain', alpha=a) #color indicates chain
        sns.scatterplot(data = loc_subset, x = 'deathX', y = 'deathY', marker = "+", hue ='num_chain', alpha=a) #marker type indicates birth/death 
        plt.xlabel('x')
        plt.ylabel('y')
    #     plt.ylim(-30000, 30000)
    #     plt.xlim(-50000, 100000)

        plt.title(sub_version)

#         out_file = 'plots/migration2_v%s.png' % sub_version
#         plt.savefig(out_file)

In [ ]:
for s in np.arange(252,302):
    plot_individual_immigration(s)
    plt.show()